In [13]:
import os
import scipy.misc
import numpy as np
import tensorflow as tf
import pickle as pkl
import random

In [14]:
pokemon = []
path = "pokemon/global-link/"
for file in os.listdir(path):
    if file.endswith('.png'):
        if file.startswith('.'):
            continue
        pic_path = path + str(file)                                                                        
        im = scipy.misc.imread(pic_path, flatten=False, mode='RGB')
        im = scipy.misc.imresize(im,(72,72))
        
        image = np.asarray(im)
        pokemon.append(image)
print(len(pokemon))

939


In [15]:
def batch(data, batch_size):
    random.shuffle(data)
    for index in range(0, len(data), batch_size):
        yield data[index: index + batch_size]

In [16]:
def model_inputs(image_width, image_height, image_channels, z_dim):

    real_input = tf.placeholder(tf.float32, [None, image_width, image_height, image_channels])
    Z = tf.placeholder(tf.float32, [None, z_dim])

    return real_input, Z

In [17]:
def generator(z,reuse=False, alpha=0.1,epsilon=1e-3):
    
    with tf.variable_scope('generator', reuse=reuse):
 
        batch_size = tf.shape(z)[0]
        w_ = tf.get_variable('gener_a_', [100, 325*9*9])
        w0 = tf.get_variable('gener_a', [3, 3, 512, 9*9*13], initializer = tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        w1 = tf.get_variable('gener_a1', [3, 3, 256, 512], initializer = tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        w2 = tf.get_variable('gener_a2', [3, 3, 128, 256], initializer = tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        w3 = tf.get_variable('gener_a3', [3, 3, 3, 128], initializer = tf.contrib.layers.xavier_initializer(), dtype=tf.float32)     
        b1 = tf.get_variable('b1', [512], initializer = tf.zeros_initializer())
        b2 = tf.get_variable('b2', [256], initializer = tf.zeros_initializer())
        b3 = tf.get_variable('b3', [128], initializer = tf.zeros_initializer())
        b4 = tf.get_variable('b4', [3], initializer = tf.zeros_initializer())


        scale1 = tf.get_variable('sc1', [512], initializer= tf.ones_initializer())
        beta1 = tf.get_variable('bet1', [512], initializer=tf.zeros_initializer())
        scale2 = tf.get_variable('sc2', [256], initializer= tf.ones_initializer())
        beta2 = tf.get_variable('bet2', [256], initializer=tf.zeros_initializer())
        scale3 = tf.get_variable('sc3', [128], initializer= tf.ones_initializer())
        beta3 = tf.get_variable('bet3', [128], initializer=tf.zeros_initializer())


        h0 = tf.matmul(z, w_)
        h0 = tf.reshape(h0, [batch_size, 5, 5, 9*9*13])

        h1 = tf.nn.conv2d_transpose(h0, w0, [batch_size, 9, 9, 512], strides=[1,2,2,1], padding='SAME')
        h1 = tf.add(h1, b1)
        batch_mean1, batch_var1 = tf.nn.moments(h1,[0])
        h1 = tf.nn.batch_normalization(h1, batch_mean1, batch_var1, beta1, scale1, epsilon)
        h1 = tf.nn.leaky_relu(h1, alpha) 

        h2 = tf.nn.conv2d_transpose(h1, w1, [batch_size, 18, 18, 256], strides=[1,2,2,1], padding='SAME' )
        h2 = tf.add(h2, b2)
        batch_mean2, batch_var2= tf.nn.moments(h2,[0])
        h2 = tf.nn.batch_normalization(h2, batch_mean2, batch_var2, beta2, scale2, epsilon)
        h2 = tf.nn.leaky_relu(h2, alpha) 

        h3 = tf.nn.conv2d_transpose(h2, w2, [batch_size, 36, 36, 128], strides=[1,2,2,1], padding='SAME')
        h3 = tf.add(h3, b3)
        batch_mean3, batch_var3 = tf.nn.moments(h3,[0])
        h3 = tf.nn.batch_normalization(h3, batch_mean3, batch_var3, beta3, scale3, epsilon)            
        h3 = tf.nn.leaky_relu(h3, alpha)  

        logits = tf.nn.conv2d_transpose(h3, w3, [batch_size, 72, 72, 3], strides=[1,2,2,1], padding='SAME')       
        logits = tf.add(logits, b4)

        out = tf.tanh(logits)

        return out, logits

In [18]:
def discriminator(images, reuse=False, alpha=0.1, dropout=0.5):
    with tf.variable_scope('discriminator', reuse=reuse):
        h1 = tf.layers.conv2d(images, 128, 3, strides=1, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.zeros_initializer())
        h1 = tf.nn.avg_pool(h1, [1,2,2,1], [1,2,2,1], padding='SAME')
        h1 = tf.nn.dropout(h1, dropout)
        h1 = tf.nn.leaky_relu(h1, alpha)
        
        h2 = tf.layers.conv2d(h1, 256, 3, strides=1, padding='SAME' , kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.zeros_initializer())
        h1 = tf.nn.avg_pool(h2, [1,2,2,1], [1,2,2,1], padding='SAME')
        h2 = tf.nn.leaky_relu(h2, alpha)
        
        h3 = tf.layers.conv2d(h2, 512, 3, strides=1, padding='SAME' , kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.zeros_initializer())
        h3 = tf.nn.avg_pool(h3, [1,2,2,1], [1,2,2,1], padding='SAME')
        h3 = tf.nn.leaky_relu(h3, alpha)
        
        flat = tf.reshape(h3, (-1, 9*9*512))
        flat = tf.nn.dropout(flat, 0.5)
        
        logits = tf.layers.dense(flat, 1 , kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.zeros_initializer())
        out = tf.sigmoid(logits)

        return out, logits

In [19]:
input_size = pokemon[0].shape[0] * pokemon[0].shape[1]
z_dim = 100
out_channel_dim = 3
h_dim = 72
w_dim = 72
smoothing = 0.1
batch_size = 32
epochs = 200
print("done..")

done..


In [20]:
tf.reset_default_graph()
input_real, input_z = model_inputs(w_dim, h_dim, out_channel_dim, z_dim)
g_model, g_logits = generator(input_z)
d_model_real, d_logits_real = discriminator(input_real)
d_model_fake, d_logits_fake = discriminator(g_model, reuse=True)
tf.summary.image('output_image',g_model)
print("done...")

done...


In [21]:
mean_diff = 0
var_diff = 0
d_loss_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_model_real)*(1-smoothing))) 
d_loss_fake = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_model_fake)))
g_loss_mod = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_model_fake)))

g_loss = g_loss_mod + mean_diff + var_diff
d_loss = d_loss_real + d_loss_fake

In [ ]:
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if var.name.startswith('discrim')]
g_vars = [var for var in t_vars if var.name.startswith('gene')]

#with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): 
g_train_opt = tf.train.AdamOptimizer(0.0002).minimize(g_loss, var_list=g_vars)
d_train_opt = tf.train.AdamOptimizer(0.0001).minimize(d_loss, var_list=d_vars)

In [ ]:
samples = []
losses = []
var_losses = []
mean_losses = []
saver = tf.train.Saver(var_list=g_vars)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    graph_def = sess.graph
    tf_tensorboard = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter("./checkpoints/train_store_conv_lstm",graph_def)
    summary_writer_it = 0

    for e in range(epochs):
        for ii in range(len(pokemon)//batch_size):
            poke_batch = next(batch(pokemon, batch_size))
            poke_batch = ((np.array(poke_batch) / 255.) - 0.5) * 2
            batch_z = np.random.uniform(-1, 1, size=(batch_size, z_dim))
            print(batch_z.shape)
            _ = sess.run(g_train_opt, feed_dict={input_real: poke_batch, input_z: batch_z})
            _ = sess.run(d_train_opt, feed_dict={input_real: poke_batch, input_z: batch_z})
            summary = sess.run(tf_tensorboard, feed_dict={input_real: poke_batch, input_z: batch_z})
            summary_writer.add_summary(summary, summary_writer_it)
            summary_writer_it += 1

        
        train_loss_g = g_loss_mod.eval({input_z: batch_z, input_real: poke_batch})
        train_loss_d = sess.run(d_loss, {input_z: batch_z, input_real: poke_batch})
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g)) 
        #var_diff1 = var_diff.eval(session=sess, feed_dict={input_real: poke_batch, input_z: batch_z})
        #mean_diff1 = mean_diff.eval(session=sess, feed_dict={input_real: poke_batch, input_z: batch_z})
        #print('Var_diff: ' +str(var_diff1) + ' Mean_diff: ' +str(mean_diff1))
        losses.append((train_loss_d, train_loss_g))
        #var_losses.append(var_diff1)
        #mean_losses.append(mean_diff1)
        sample_z = np.random.uniform(-1, 1, size=(32, z_dim))
        gen_samples, _ = sess.run(
                       generator(input_z, reuse=True),
                       feed_dict={input_z: sample_z})
        samples.append(gen_samples)
        saver.save(sess, './checkpoints/generator.ckpt')

with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)

(32, 100)
(32, 100)
(32, 100)
